In [28]:
import torch
import os
import pandas as pd
import numpy as np
from torch.utils.data import Dataset

In [29]:
class myData(Dataset):
    def __init__(self, file = 'train3.csv'):
        super().__init__()
        self.data = pd.read_csv(file )
        self.n = len(self.data)
        
    def __len__(self):
        return self.n
    
    def __getitem__(self,idx):
        return (self.data.iloc[idx,0],self.data.iloc[idx,1])

myData()[1]

('上午可以配送么？', '预约时间')

In [30]:
class myDict:
    
    def __init__(self,dict_file="model_2.vec",ndim = 100):
        self.dict = {}
        self.ndim = ndim
        with open(dict_file,encoding='utf8') as f:
            f.readline()
            for line in f:
                line = line.strip().split(' ')
                self.dict[line[0]] = np.array([float(x) for x in line[1:]])
       
    def lineToTensor(self,line):
       
        line = list(line)
        dd = np.zeros((len(line),self.ndim))
        for i,c in enumerate(line):
            if c in self.dict.keys():
                dd[i,:] = self.dict[c]
            

        return torch.FloatTensor(dd)
    



# 模型

In [31]:
class RNN(torch.nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()
        self.ips = input_size
        self.hs = hidden_size
        self.os = output_size
        
        self.i2r = torch.nn.Linear(self.ips+self.hs,self.hs)
        
        self.i2h = torch.nn.Linear(self.ips,self.hs)
        self.h2o = torch.nn.Linear(self.hs, self.os)
       

    def forward(self,input, hidden):
        x = torch.cat((input,hidden),0)
        #print(x.shape)
        r = torch.sigmoid(self.i2r(x))
        #print(r.shape)
        hidden = r*hidden + (1-r)*self.i2h(input)
        #hidden = hidden / (0.0001+torch.max(torch.abs(hidden)))
        output = self.h2o(hidden)
        return (output,hidden)

rnn = RNN(100,4,3)   

In [33]:
rnn(torch.zeros(100),torch.zeros(4))

(tensor([0.1347, 0.3135, 0.4697], grad_fn=<AddBackward0>),
 tensor([-0.0478, -0.0085,  0.0019,  0.0036], grad_fn=<AddBackward0>))

In [34]:
def calc_rnn(x,nhidden = 4):
    hidden = torch.zeros(nhidden)
    for i in range(x.shape[0]):
        out,hidden = rnn(x[i],hidden)
        #print(out.data,hidden.data)
    return out


In [35]:
mydata = myData()
mydict = myDict()
ndata = len(mydata)

def sim1(a,b):
    return torch.abs(torch.sum(a*b)/torch.sqrt(torch.sum(a*a)*torch.sum(b*b)))
    
def sim2(a,b):
    d = torch.sum((a-b)**2)
    return 1/(1+d)

def calc_md(x):
    return calc_rnn(mydict.lineToTensor(x))

def train():
    out = torch.FloatTensor([0])
    for i in range(100):
        a = np.random.randint(0,39)
        b = np.random.randint(0,ndata)
        oa = calc_md(mydata[a][0])
        ob = calc_md(mydata[b][0])

        yh = sim1(oa,ob)
        if (yh>0 and yh <1):
            if mydata[a][1]== mydata[b][1]:
                out = out - torch.log(yh)
            else:
                out = out - torch.log(1-yh)
    return(out)


In [48]:
for i in range(10):
    
    out = train()
    rnn.zero_grad()
    out.backward()

    print(i , out.data)
    for p in rnn.parameters():
        p.data -= p.grad.data * 0.01

0 tensor([132.1419])
1 tensor([136.8150])
2 tensor([159.4898])
3 tensor([380.1130])
4 tensor([142.2850])
5 tensor([118.3133])
6 tensor([132.3394])
7 tensor([125.2715])
8 tensor([111.4980])
9 tensor([145.8117])


In [ ]:
yh_list


In [55]:
sim1(calc_md('今天晚上8点半就可以'),calc_md('9点可以送吗'))

tensor(0.6349, grad_fn=<AbsBackward>)

In [ ]:
calc_rnn(mydict.lineToTensor( '是的'))

In [ ]:
calc_rnn(mydict.lineToTensor( '你好，七点能送嘛？'))

In [ ]:
mydict.lineToTensor( '是的')

In [ ]:
mydict.lineToTensor( '你好，七点能送嘛？')

In [ ]:
list(rnn.named_parameters())


In [ ]:
torch.sigmoid(torch.Tensor([-0.1168]))

In [ ]:
yh_list

In [ ]:
d_list